In [ ]:
import matplotlib.pyplot as plt

import bluemira.geometry.tools
from bluemira.base.components import PhysicalComponent, GroupingComponent
import bluemira.display as display
from bluemira.display._matplotlib_plot import (
    PointsPlotter,
    WirePlotter,
    FacePlotter,
)
from bluemira.geometry.parameterisations import PrincetonD
from bluemira.geometry.face import BluemiraFace

## Setup

Creation of a closed wire and respective face

PrincetonD parametrisation is used as example.

Note: the curve is generated on the x-z plane

In [ ]:
p = PrincetonD()
p.adjust_variable("x1", 4, lower_bound=3, upper_bound=5)
p.adjust_variable("x2", 16, lower_bound=10, upper_bound=20)
p.adjust_variable("dz", 0, lower_bound=0, upper_bound=0)
wire = p.create_shape()
face = BluemiraFace(wire)

## Default plotting

We can display the BluemiraWire and BluemiraFace in the following way, using the
default settings.

In [ ]:
display.plot_2d(wire)
display.plot_3d(wire)
display.show_cad(face)

## Modifying defaults

Default plot options can be obtained in form of a dictionary instancing one of the
default plotters, e.g.:

In [ ]:
my_options = FacePlotter().options.as_dict()
# Modifying the dictionary and passing it to a plot function will display the plot
# with the new options
my_options["show_points"] = False
display.plot_2d(wire, **my_options)

# Once you get familiar with the options, you can also make your own dictionaries, and
# pass them to the plotting functions

my_options = {"show_points": False, "wire_options": {"color": "red", "linewidth": 3}}
display.plot_2d(wire, **my_options)


Discretise the wire to an array of points.

In [ ]:
points = p.create_array(n_points=10).T

## Points Plot

Simple plot of the obtained points.

A PointsPlotter is created specifying size, edge and face colors.

In [ ]:
pplotter = PointsPlotter(
    point_options={"s": 30, "facecolors": "red", "edgecolors": "black"}
)
pplotter.plot_2d(points)

## 3D Scatter Plot

A plot of the same points, but in 3D this time.

In [ ]:
pplotter.plot_3d(points)

## Wire Plot

A WirePlotter is used with the default setup with:

- plane = xz (this is the projection plane, not a section plane)
- point size = 10
- ndiscr = 10
- plot title

In [ ]:
wplotter = WirePlotter(plane="xz")
wplotter.options.point_options["s"] = 20
wplotter.options.ndiscr = 5
wplotter.plot_2d(wire)

## 3D Curve Plot

A plot of the same wire, but in 3D this time.

In [ ]:
wplotter.plot_3d(wire)

## Wire Plot with Matplotlib Default Options

In this example poptions is set to an empty dict. The default matplotlib are used.

In [ ]:
wplotter.options.point_options = {}
wplotter.plot_2d(wire)
# The plot is immediately shown by default, so it is not possible to act on the plot

## Wire plot with some modifications

In this example, we choose our own matplotlib Axes onto which to plot, disable the
automatic display of the plot (show=False), and apply a title to the plot

In [ ]:
f, ax = plt.subplots()
wplotter.options.point_options = {}
wplotter.plot_2d(wire, ax=ax, show=False)
ax.set_title("Wire plot")
plt.show()

## Face Plot

A FacePlotter is used with the default setup with:

- plane = xz (this is the projection plane, not a section plane)
- ndiscr = 30
- plot title

In [ ]:
f, ax = plt.subplots()
fplotter = FacePlotter(plane="xz")
fplotter.options.ndiscr = 30
fplotter.plot_2d(face, ax=ax, show=False)
ax.set_title("Face plot without points")
plt.show()

## Face Plot with Points Enabled

We've set the points to be disabled by default, but we can activate them again for
individual plotters.

In [ ]:
f, ax = plt.subplots()
fplotter = FacePlotter(plane="xz")
fplotter.options.ndiscr = 30
fplotter.options.show_points = True
fplotter.plot_2d(face, ax=ax, show=False)
ax.set_title("Face plot with points")
plt.show()

## Make a Second Face

A second geometry is created, surrounding our original face.

In [ ]:
p2 = PrincetonD()
p2.adjust_variable("x1", 3.5, lower_bound=3, upper_bound=5)
p2.adjust_variable("x2", 17, lower_bound=10, upper_bound=20)
p2.adjust_variable("dz", 0, lower_bound=0, upper_bound=0)
wire2 = p2.create_shape()
face2 = BluemiraFace(wire2)

## Combined Face Plot

Face and face2 are plotted using the same FacePlotter. Since no plot options have
been changed, the two faces will be plotted in the same way (e.g. same color).

In [ ]:
fplotter2 = FacePlotter(plane="xz")
fplotter2.options.show_points = True
fplotter2.options.face_options = {"color": "blue"}

f, ax = plt.subplots()
fplotter2.plot_2d(face, ax=ax, show=False)
fplotter2.plot_2d(face2, ax=ax, show=False)
ax.set_title("Both faces in blue")
plt.show()
print(f"fplotter2.options: {fplotter2.options}")

## Combined Face Plot with Different Colours

Plot both face with different colour.

Note: if face is plotted before face2, face2 will be "covered" by face.

In [ ]:
f, ax = plt.subplots()

fplotter2.options.face_options = {"color": "blue"}
fplotter2.plot_2d(face2, ax=ax, show=False)
fplotter2.options.face_options = {"color": "green"}
fplotter2.plot_2d(face, ax=ax, show=False)
ax.set_title("Both faces with different colors")
plt.show()

## Face with Hole

A third face is create as difference between face and face2 (a BluemiraFace object
has been created using wire2 as outer boundary and wire as inner boundary).

Note:
- when plotting points, it can happen that markers are not centred properly as
      described in https://github.com/matplotlib/matplotlib/issues/11836
- face3 is created with a wire deepcopy in order to be able to modify face and face2
(and thus wire and wire2) without modifying face3

In [ ]:
face3 = BluemiraFace([wire2.deepcopy(), wire.deepcopy()])
fplotter3 = FacePlotter(plane="xz")
fplotter3.options.show_points = True
fplotter3.plot_2d(face3, show=False)
fplotter3.ax.set_title("Face with hole - points enabled")
fplotter3.show()

fplotter3.options.face_options["color"] = "blue"
fplotter3.options.show_points = False
fplotter3.plot_2d(face3, show=False, ax=None)
fplotter3.ax.set_title("Face with hole - points disabled - blue")
fplotter3.show()

## Perform Some Face Operations

Scale and move our face

In [ ]:
bari = face.center_of_mass
face.scale(0.5)
new_bari = face.center_of_mass
diff = bari - new_bari
v = (diff[0], diff[1], diff[2])
face.translate(v)

## Wires and Faces

Create and plot a couple of Wires and then create and plot the corresponding Faces.

In [ ]:
points = [[0, 0, 0], [1, 0, 0], [1, 0, 3], [0, 0, 3]]
wire = bluemira.geometry.tools.make_polygon(points, closed=True)
wire1 = wire.deepcopy()
wire1.translate((3, 0, 5))
wplotter.plot_2d(wire, show=False)
wplotter.ax.set_title("wire")
wplotter.show()

wplotter.plot_2d(wire1, show=False)
wplotter.ax.set_title("wire1")
wplotter.show()

## Plots with Matplotlib Default Point Options

Plot the points on a boundary of a face with matplotlib defaults.

Note that, since point_options = {}, points color is automatically changed by
matplotlib.

In [ ]:
wface = BluemiraFace(wire)
w1face = BluemiraFace(wire1)
wplotter.plot_2d(wface.boundary[0])
print(f"test_boundary wplotter options: {wplotter.options}")
wplotter.plot_2d(w1face.boundary[0], ax=wplotter.ax)
print(f"test_boundary wplotter options: {wplotter.options}")
wplotter.ax.set_title("test boundary from faces - matplotlib default point_options")
wplotter.show()

## Plot with Matplotlib Default Wire Options

Plot the boundary of a face with matplotlib defaults.

Note that, since wire_options = {}, wire color is automatically changed by matplotlib

In [ ]:
wplotter.options.wire_options = {}
wplotter.plot_2d(wface.boundary[0])
print(f"test_boundary wplotter options: {wplotter.options}")
wplotter.plot_2d(w1face.boundary[0], ax=wplotter.ax)
print(f"test_boundary wplotter options: {wplotter.options}")
wplotter.ax.set_title(
    "test boundary from faces - matplotlib default point_options and wire_options"
)
wplotter.show()

## PhysicalComponent Plot

Creates a `PhysicalComponent` and plots it in the xz plane

In [ ]:
c = PhysicalComponent("Comp", face)
c.plot_2d_options.plane = "xz"
c.plot_2d_options.ndiscr = 30
ax = c.plot_2d(show=False)
ax.set_title("test component plot")
plt.show(block=True)

## GroupingComponent Plot

Creates a `GroupingComponent` and plots it in the xz plane using matplotlib defaults.
Here we override some defaults and make our custom set of plot options.

In [ ]:
my_group_options = FacePlotter().options.as_dict()
my_group_options["wire_options"] = {}
my_group_options["face_options"] = {"color": "red"}
group = GroupingComponent("Components")
c1 = PhysicalComponent("Comp1", face, parent=group)
c2 = PhysicalComponent("Comp2", wface, parent=group)
c3 = PhysicalComponent("Comp3", w1face, parent=group)
group.plot_2d(**my_group_options)

## Component and BluemiraGeo Combined Plot

Plots a component on the same axes as a BluemiraFace.

In [ ]:
wplotter.options.wire_options["color"] = "red"
ax = wplotter.plot_2d(wface.boundary[0])
fplotter.options.face_options["color"] = "green"
fplotter.options.wire_options["color"] = "black"
ax = fplotter.plot_2d(w1face, ax=ax)
ax = c.plot_2d(ax=ax)
ax.set_title("test component + bluemirageo plot")
plt.show(block=True)

Show the options from our combined plot

In [ ]:
print(f"wire plotter options: {wplotter.options}")
print(f"face plotter options: {fplotter.options}")
print(f"component plotter options: {c.plot_2d_options}")

## CAD Display

Displays a GroupingComponent in a bluemira display window.

In [ ]:
group.show_cad()

# We can also change the appeare of individual components inside the group
c1.displayer_cad_options.modify(**{"color": (0.1, 0.1, 0.1)})
c2.displayer_cad_options.modify(**{"color": (0.3, 0.2, 0.6)})
c3.displayer_cad_options.modify(**{"color": (0.2, 0.6, 0.1), "transparency": 0.5})

group.show_cad()